

- Step‑by‑step execution cells  
- Big, colorful instructional Markdown for students  
- Commands for Ollama, FAISS, Streamlit, and Cloudflare Tunnel  

This notebook doesn’t run the Streamlit server inside Jupyter (since `streamlit run` is external), but gives **clear instructions + copyable commands**.  

---

# 📘 Hospital_Chatbot_Cloudflare_Guide.ipynb

```python
# ================================================================
# 🏥 Hospital RAG Chatbot Deployment with FAISS + Ollama + Cloudflare
# ================================================================
# 🎓 Student-Friendly Guide with step-by-step colorful instructions
# ================================================================

# ⚠️ IMPORTANT:
# 1. This Notebook is for building FAISS index & guiding deployment.
# 2. The Streamlit app (`hospital_chat_app.py`) must be launched separately in terminal.
# ================================================================
```

---

## 📌 Section 1 – Install Dependencies

```markdown
# 🌈 Step 1 – Install All Requirements

Run the following cell to install Streamlit, FAISS, LangChain modules,
Ollama connector, and Sentence-Transformers for embeddings.
```

```python
!pip install streamlit faiss-cpu langchain langchain_community langchain-ollama sentence-transformers
```

---

## 📌 Section 2 – Setup Ollama

```markdown
# 🌈 Step 2 – Install & Setup Ollama

1. Download Ollama 👉 https://ollama.ai
2. Open terminal (not Jupyter) and run:
   ```bash
   ollama serve
   ```
3. Pull small, fast models:
   ```bash
   ollama pull phi3:mini
   ollama pull gemma:2b
   ollama pull mistral
   ```

⚡ For class demos, `phi3:mini` is the fastest choice!
```

---

## 📌 Section 3 – Create Hospital Knowledge Base

```markdown
# 🌈 Step 3 – Build Our Hospital Knowledge Base
We'll use JSON files with data on hospital departments, services, and visiting rules.
```

```python
import os, json

os.makedirs("hospital_data", exist_ok=True)

hospital_segments = {
    "departments.json": [
        {"topic": "Cardiology",
         "content": "Our Cardiology department specializes in diagnosis and treatment of heart diseases such as ECG, echocardiography, angioplasty, and cardiac surgery."},
        {"topic": "Pediatrics",
         "content": "Pediatrics provides care for infants and adolescents, including immunizations, growth monitoring, and treatment of childhood illnesses."}
    ],
    "services.json": [
        {"topic": "Emergency",
         "content": "The Emergency Department operates 24/7 handling trauma, strokes, heart attacks, and urgent medical cases with rapid response teams."},
        {"topic": "OPD",
         "content": "Outpatient Department (OPD) handles consultations, diagnostics, and follow-ups without hospital admission."}
    ],
    "guidelines.json": [
        {"topic": "Visiting Hours",
         "content": "Patients may be visited between 4–7PM. Children under 12 are not allowed in ICU wards."}
    ]
}

for fname, values in hospital_segments.items():
    with open(os.path.join("hospital_data", fname), "w") as f:
        json.dump(values, f, indent=2)

print("✅ Hospital dataset created in /hospital_data")
```

---

## 📌 Section 4 – Build FAISS Vector Index

```markdown
# 🌈 Step 4 – Create FAISS Index for Knowledge Retrieval
We'll embed texts using `all-mpnet-base-v2` and save index as "hospital_faiss_index".
```

```python
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

all_texts = []
for fname in os.listdir("hospital_data"):
    with open(os.path.join("hospital_data", fname)) as f:
        docs = json.load(f)
        for d in docs:
            all_texts.append(d["content"])

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = splitter.create_documents(all_texts)

embedding = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

vectordb = FAISS.from_documents(docs, embedding)
vectordb.save_local("hospital_faiss_index")

print("✅ FAISS index created and saved as 'hospital_faiss_index/'")
```

---

## 📌 Section 5 – Test Retrieval

```markdown
# 🌈 Step 5 – Test the Knowledge Base Retrieval
We query FAISS manually (before connecting Ollama).
```

```python
vectordb = FAISS.load_local("hospital_faiss_index", embedding, allow_dangerous_deserialization=True)
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k":2})

docs = retriever.get_relevant_documents("What are the visiting hours?")
for d in docs:
    print("🔍 Retrieved:", d.page_content)
```

---

## 📌 Section 6 – Run Streamlit App (Hospital Chatbot)

```markdown
# 🌈 Step 6 – Run the Chatbot Locally

⚙️ Save your Streamlit app as **hospital_chat_app.py** (already prepared).
Launch it in Terminal, not Jupyter:

```bash
streamlit run hospital_chat_app.py
```

This starts your chatbot at:
➡ http://localhost:8501
```

---

## 📌 Section 7 – Expose Publicly with Cloudflare

```markdown
# 🌈 Step 7 – Share Your Chatbot with the World (FREE using Cloudflare)

### 1. Install Cloudflare Tunnel
- Download `cloudflared` for Windows from:
  👉 https://developers.cloudflare.com/cloudflare-one/connections/connect-apps/install-and-setup/installation/#windows
- Place `cloudflared.exe` in `C:\cloudflared\`
- Add that folder to PATH (Environment Variables → Path → Add `C:\cloudflared\`)

### 2. Test Installation
```powershell
cloudflared --version
```

### 3. Run Tunnel for Your Chatbot
Keep Streamlit running, then open new PowerShell:
```powershell
cloudflared tunnel --url http://localhost:8501
```

### 4. Share the Link
Cloudflare shows output:
```
INFO Visit your app at https://green-lion-1234.trycloudflare.com
```
🎉 Share this link with students so they can access it instantly from their browser or phone!
```

---

## 📌 Section 8 – Recommended Models for Classroom

```markdown
# 🌈 Recommended Models for Classroom Use (via Ollama)
1. ⚡ `phi3:mini` → fastest, ideal for classroom demos.
2. 🤏 `gemma:2b` → small, balanced quality.
3. 🧠 `mistral` → better answers, but slower.

👉 Always pull before use:
```bash
ollama pull phi3:mini
ollama pull gemma:2b
ollama pull mistral
```
```

---

# 🎯 Notebook Summary

✅ You now have a **hospital chatbot RAG pipeline** that:
- Builds & saves a FAISS knowledge base  
- Runs locally in Streamlit  
- Gets exposed to the internet **FREE** with Cloudflare Tunnel  

---

✨ This Notebook is now a **cheatsheet + hands‑on coding exercise** for your students.  

👉 Would you like me to **export this into an actual `.ipynb` file** (JSON format) so you can download and directly open it in Jupyter without copy/pasting?